# finetuning for summarization task the t5

In [1]:
# from huggingface_hub import notebook_login

# notebook_login()

In [2]:
import pandas as pd
import os

EUR_LEX_SUM_PATH = "../data/raw/summarization/eur-lex-sum_it/"
TRAIN_EUR_PATH = os.path.join(EUR_LEX_SUM_PATH, "train.json")
TEST_EUR_PATH = os.path.join(EUR_LEX_SUM_PATH, "test.json")

# Read JSON file line by line
with open(TRAIN_EUR_PATH, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Join the lines into a single JSON string
json_data_train = "[" + ",".join(lines) + "]"

# Load JSON into Pandas DataFrame
try:
    data_train = pd.read_json(json_data_train)
    print(data_train)
except ValueError as e:
    print("Error:", e)


with open(TEST_EUR_PATH, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Join the lines into a single JSON string
json_data_test = "[" + ",".join(lines) + "]"

# Load JSON into Pandas DataFrame
try:
    data_test = pd.read_json(json_data_test)
    print(data_test)
except ValueError as e:
    print("Error:", e)

/tmp/ipykernel_1467555/1646270494.py:17: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_train = pd.read_json(json_data_train)


            celex_id                                          reference  \
0         32019R0980  21.6.2019\nIT\nGazzetta ufficiale dell'Unione ...   
1         32019D0785  16.5.2019\nIT\nGazzetta ufficiale dell'Unione ...   
2         32019R1122  2.7.2019\nIT\nGazzetta ufficiale dell'Unione e...   
3         32019R0856  28.5.2019\nIT\nGazzetta ufficiale dell'Unione ...   
4     22020A0724(01)  24.7.2020\nIT\nGazzetta ufficiale dell’Unione ...   
...              ...                                                ...   
1023  21986A0618(01)  EUR-Lex - 21986A0618(01) - IT\nAvis juridique ...   
1024      32004R0785  30.4.2004\nIT\nGazzetta ufficiale dell'Unione ...   
1025      31965R0019  EUR-Lex - 31965R0019 - IT\nAvis juridique impo...   
1026  31958R0003(01)  EUR-Lex - 31958R0003(01) - IT\nAvis juridique ...   
1027  22004A0210(01)  EUR-Lex - 22004A0210(01) - IT\nAvis juridique ...   

                                                summary  
0     Prospetto da pubblicare per l’offer

/tmp/ipykernel_1467555/1646270494.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_test = pd.read_json(json_data_test)


           celex_id                                          reference  \
0        32019R1156  12.7.2019\nIT\nGazzetta ufficiale dell'Unione ...   
1        32019R1020  25.6.2019\nIT\nGazzetta ufficiale dell'Unione ...   
2        32019L0789  17.5.2019\nIT\nGazzetta ufficiale dell'Unione ...   
3        32019R0515  29.3.2019\nIT\nGazzetta ufficiale dell'Unione ...   
4        32019R0517  29.3.2019\nIT\nGazzetta ufficiale dell'Unione ...   
..              ...                                                ...   
183      32008R0762  13.8.2008\nIT\nGazzetta ufficiale dell’Unione ...   
184      32010R1236  31.12.2010\nIT\nGazzetta ufficiale dell'Unione...   
185      32010R0066  30.1.2010\nIT\nGazzetta ufficiale dell’Unione ...   
186      32009R0469  16.6.2009\nIT\nGazzetta ufficiale dell'Unione ...   
187  22009A0610(01)  10.6.2009\nIT\nGazzetta ufficiale dell’Unione ...   

                                               summary  
0    Distribuzione transfrontaliera degli organismi...

In [3]:
from datasets import Dataset

dataset_train = Dataset.from_pandas(data_train[:300])
dataset_test = Dataset.from_pandas(data_test[:50])

## code for finetuning

In [4]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
if model_checkpoint in [
    "t5-small",
    "t5-base",
    "t5-larg",
    "t5-3b",
    "t5-11b",
    "google/flan-t5-base",
]:
    prefix = "summarize: "
else:
    prefix = ""

In [7]:
import torch

print(torch.cuda.is_available())

True


In [8]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["reference"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(
        text_target=examples["summary"], max_length=max_target_length, truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_datasets_train = dataset_train.map(preprocess_function, batched=True)
tokenized_datasets_test = dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [11]:
from transformers import (
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [12]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    # predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
# import nltk
# import numpy as np

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Rouge expects a newline after each sentence
#     decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

#     # Note that other metrics may not have a `use_aggregator` parameter
#     # and thus will return a list, computing a metric for each sentence.
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
#     # Extract a few results
#     result = {key: value * 100 for key, value in result.items()}

#     # Add mean generated length
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)

#     return {k: round(v, 4) for k, v in result.items()}

In [15]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    data_collator=data_collator,
    tokenizer=tokenizer
    # compute_metrics=compute_metrics
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 20.00 GiB total capacity; 4.51 GiB already allocated; 522.75 MiB free; 4.63 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF